In [1]:
import numpy as np
import string
import Image
import time
import datetime
import os
from scipy import misc

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

import emotion_model
import dwdii_transforms

Using Theano backend.


In [2]:
imagePath = "/root/facial_expressions/images"
dataPath = "/root/facial_expressions/data/legend.csv"

In [3]:
os.listdir('/root/facial_expressions/data')

['legend.csv', '500_picts_satz.csv']

### Load Training and Test Data

In [4]:
#emoMetaData = dwdii_transforms.load_training_metadata(dataPath, True)

In [5]:
#len(emoMetaData)

In [6]:
maxData = 1820
X_data, Y_data = dwdii_transforms.load_data(dataPath, imagePath, maxData = maxData, verboseFreq = 200)
print X_data.shape
print Y_data.shape

0.000000: Jean_Chretien_0048.jpg
0.109890: Steve_Case_0001.jpg
0.219780: Josh_Evans_0001.jpg
0.329670: Kim_Clijsters_0007.jpg
0.439560: Jayamadhuri_180.jpg
0.439560: Mike_Myers_0004.jpg
0.549451: Samantha_Ledster_0001.jpg
0.659341: Gonzalo_Sanchez_de_Lozada_0005.jpg
0.769231: Scott_Sullivan_0002.jpg
0.879121: George_W_Bush_0181.jpg
0.989011: Mike_Weir_0002.jpg
(1820, 350, 350)
(1820, 1)


### Transformations
In this section, we will apply transformations to the existing images to increase of training data, as well as add a bit of noise in the hopes of improving the overall training activities.

In [13]:
np.zeros([X_data.shape[0] * 2, 350, 350])

MemoryError: 

In [8]:
newImgs = np.zeros([X_data.shape[0] * 2, X_data.shape[1], X_data.shape[2]])
newYs = np.zeros([Y_data.shape[0] * 2, Y_data.shape[1]], dtype=np.int8)
print newImgs.shape
print newYs.shape

MemoryError: 

In [ ]:
ndx = 0
for i in range(X_data.shape[0]):
    img = X_data[i]
    #print img[:]
    #im = Image.fromarray(img)
    #print img.shape
    #s = list(img.shape)
    #print len(s)
    
    #fig = plt.figure(figsize=(350,350))
    #plt.imshow(img)
    #plt.axis('off')
    #plt.tight_layout()  
    
    #fig.savefig("test.png", dpi=fig.dpi)
    #plt.close(fig)
    
    #img *= 255.0
    #img = img.astype("int8")
    
    
    img1 = dwdii_transforms.reflectY(img)
    newImgs[ndx] = img1
    newYs[ndx] = Y_data[i]
    misc.imsave("test1.png", img1)
    ndx += 1
    
    img2 = dwdii_transforms.rotate5(img)
    newImgs[ndx] = img2
    newYs[ndx] = Y_data[i]
    misc.imsave("test2.png", img2)
    ndx += 1

    #break
    
print("Done", str(datetime.datetime.now()))

('Done', '2016-11-23 12:23:03.227427')


In [ ]:
X_data2 = np.concatenate((X_data, newImgs))
Y_data2 = np.concatenate((Y_data, newYs))
print X_data2.shape
print Y_data2.shape

### Split Training/Test Sets

The following code segment splits the data into training and test data sets. Currently this is just a standard 80/20 split for training and test respectively. A random.shuffle should be probably be added here as well.

In [24]:
# Split the data into Training and Test sets
trainNdx = int(X_data2.shape[0] * .8)
print trainNdx
X_train, X_test = np.split(X_data2, [trainNdx])
Y_train, Y_test = np.split(Y_data2, [trainNdx])
print X_train.shape
print X_test.shape

print Y_train.shape
print Y_test.shape

240
(240, 350, 350)
(60, 350, 350)
(240, 1)
(60, 1)


### Define the Model
In this section, we define the model. The `emotion_model` module contains the model definition itself. `emotion_model_v1` is a basic convolutional neural network.

In [25]:
# Map the emotions to integers for categorization later.
emotions = dwdii_transforms.emotionNumerics()
print emotions
print len(emotions)

{'sadness': 6, 'neutral': 2, 'contempt': 7, 'disgust': 1, 'anger': 0, 'surprise': 4, 'fear': 5, 'happiness': 3}
8


In [26]:
model = emotion_model.emotion_model_v6(len(emotions), verbose=True)

Compiling Model ... 
____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
zeropadding2d_2 (ZeroPadding2D)    (None, 1, 360, 360) 0           zeropadding2d_input_2[0][0]      
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)    (None, 32, 331, 331)28832       zeropadding2d_2[0][0]            
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)      (None, 32, 165, 165)0           convolution2d_2[0][0]            
____________________________________________________________________________________________________
flatten_2 (Flatten)                (None, 871200)      0           maxpooling2d_2[0][0]             
______________________________________________________________________

### Training the Model

The following code segment trains the model using the `run_network` helper function. I seem to be hitting a memory issue (my interpretation), when I have batches above a certain threshold. Batches=10 work fine, but batches of 100 are too big. May need to allocate more RAM to the docker container.

In [27]:
# Reshape to the appropriate shape for the CNN input
testX = X_test.reshape(X_test.shape[0], 1, 350, 350)
trainX = X_train.reshape(X_train.shape[0], 1, 350, 350)

In [28]:
print "Training start: " + str(datetime.datetime.now())
m, h = emotion_model.run_network([trainX, testX, Y_train, Y_test], model, batch=32, epochs=1)

Training start: 2016-11-23 12:11:48.803543
(240, 7)
(60, 7)
Training model...


Exception: A target array with shape (240, 7) was passed for an output of shape (None, 8) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.

* Model v2: Network's test score [loss, accuracy]: [9.1067240715026863, 0.435] - 4474s 
  * #2: Network's test score [loss, accuracy]: [9.1067240715026863, 0.435] - 5469s 
* Model v3: Network's test score [loss, accuracy]: [8.0187525367736825, 0.50249999999999995] - 2499s
* Model v4.1: Network's test score [loss, accuracy]: [9.1067240715026863, 0.435] - 2137s 
* Movel v4.2: Network's test score [loss, accuracy]: [8.0187525367736825, 0.50249999999999995] - 3108s 
* Model v5: Network's test score [loss, accuracy]: [6.9666682052612305, 0.19500000000000001] - 1682s 
* Model v6: Network's test score [loss, accuracy]: [1.7120025205612182, 0.23749999999999999] - 3020s 
* Model v7: Network's test score [loss, accuracy]: [7.9999716758728026, 0.14000000000000001] - 4610s 
* Model v8: Network's test score [loss, accuracy]: [1.6948303937911988, 0.19500000000000001] - 3313s 
* Model v6 w/ flatten: Network's test score [loss, accuracy]: [7.1107604598999021, 0.17249999999999999] - 3044s 

In [ ]:
model.save_weights("dwdii-emo-01v6.hdf5", overwrite=True)

### References

* OpenCV/CV2: http://askubuntu.com/questions/447409/how-to-install-opencv-2-9-for-python
* Docker Commit: http://stackoverflow.com/questions/19585028/i-lose-my-data-when-the-container-exits
  * docker ps -l
  * docker commit <ContainerID> dittenhafer/dl